In [68]:
import pandas as pd
from tqdm import tqdm
from glob import glob
import logging
import os
import re
from sklearn.model_selection import train_test_split

In [69]:
def get_graph_type(s):
    s = s.split('/')[-1]
    s = s.split('_')[:-1]
    s = "_".join(s)
    return s

In [70]:
def get_files_df(INPUT_GRAPHS_FOLDER, INFERENCE_GRAPHS_FOLDER):
    logging.info("Creating dataframe for OWL2DL-1 input/inference pairs")
    rdf_files = []
    for input_graph_path in tqdm(sorted(glob(INPUT_GRAPHS_FOLDER + "*"))):
        input_graph_file = os.path.basename(input_graph_path)
        inference_path = INFERENCE_GRAPHS_FOLDER + input_graph_file.replace('.owl','_inferred.owl')
        graph_type = get_graph_type(input_graph_path)
        rdf_pair = {"input_graph_file": input_graph_path, "inference_file": inference_path, "graph_type": graph_type}
        rdf_files.append(rdf_pair)
    files_df = pd.DataFrame.from_dict(rdf_files)
    return files_df

In [71]:
files_df = get_files_df('MyJenaProject/input/', 'MyJenaProject/output/')

100%|██████████| 3662/3662 [00:00<00:00, 226488.46it/s]


In [72]:
df_count = pd.DataFrame(files_df['graph_type'].value_counts())
graph_type_2_keep = df_count[df_count['count'] > 1].index
files_df = files_df[files_df['graph_type'].isin(graph_type_2_keep)]

In [73]:
def train_validate_test_split(df, train_percent=0.6, validate_percent=0.2, stratify=None, seed=1):
    val_test_percent = 1 - train_percent
    test_percent = (1 - (train_percent + validate_percent))
    test_percent = test_percent / (test_percent + validate_percent)
    if stratify:
        df_train, df_val_test = train_test_split(df, test_size=val_test_percent, random_state=seed,
                                                 stratify=df[stratify])
        df_val, df_test = train_test_split(df_val_test, test_size=test_percent, random_state=seed,
                                           stratify=df_val_test[stratify])
    else:
        df_train, df_val_test = train_test_split(df, test_size=val_test_percent, random_state=seed)
        df_val, df_test = train_test_split(df_val_test, test_size=test_percent, random_state=seed)
    return df_train, df_val, df_test

In [74]:
rdf_data_train, rdf_data_val, rdf_data_test = train_validate_test_split(files_df,
                                                                        train_percent=0.6,
                                                                        validate_percent=0.2,
                                                                        stratify="graph_type",
                                                                        seed=1)

In [76]:
def write_nt_file(data, output_file):
    nt_files_orig = data['input_graph_file']
    nt_files_inferred = data['inference_file']
    nt_files = pd.concat([nt_files_orig, nt_files_inferred])

    with open(output_file, 'w') as outfile:
        for nt_file in nt_files:
            with open(nt_file, 'r') as infile:
                outfile.write(infile.read())
                outfile.write("\n")

In [77]:
write_nt_file(rdf_data_train, 'datasets/bin/OWL2DL-1_train.owl')
write_nt_file(rdf_data_val, 'datasets/bin/OWL2DL-1_val.owl')
write_nt_file(rdf_data_test, 'datasets/bin/OWL2DL-1_test.owl')